以下、整理した実装

In [19]:
import pydeck as pdk
from plateaukit import load_dataset
import geopandas as gpd
import pandas as pd, numpy as np
from shapely.geometry import box, mapping

# ===== パラメータ =====
CENTER_ADDR = "東京都千代田区丸の内1-9-1"  # 東京駅 丸の内口
SIZE_M = [1000, 1000]                       # 表示範囲 [m]（1000m四方）
MAP_SIZE = 900                               # 表示サイズ（px）← 正方形で見やすい

# ===== データ読み込み（千代田区）＆範囲切り出し =====
ds = load_dataset("plateau-13101-chiyoda-ku-2023")
area = ds.area_from_address(CENTER_ADDR, size=SIZE_M)
gdf  = area.gdf.copy()

# 高さ（measuredHeight 優先。なければ storeys×3m で近似）
if "measuredHeight" in gdf.columns:
    h = pd.to_numeric(gdf["measuredHeight"], errors="coerce").fillna(0.0)
else:
    cand = [c for c in gdf.columns if isinstance(c,str) and "storeys" in c.lower()]
    h = (pd.to_numeric(gdf[cand[0]], errors="coerce").fillna(0.0)*3.0) if cand else pd.Series(0.0, index=gdf.index)

# WGS84（pydeck は経緯度）
gdf_ll = gdf.to_crs(4326) if gdf.crs and gdf.crs.to_epsg()!=4326 else gdf

# 表示範囲と“水面ポリゴン”（でかい青い板）
minx, miny, maxx, maxy = gdf_ll.total_bounds
pad = 0.001
water_poly = box(minx-pad, miny-pad, maxx+pad, maxy+pad)
water_geojson = {
    "type":"FeatureCollection",
    "features":[{"type":"Feature","geometry": mapping(water_poly),
                 "properties":{"color":[31,119,180,90]}}]  # 半透明の青
}
layer_water = pdk.Layer(
    "GeoJsonLayer", water_geojson,
    stroked=False, filled=True,
    get_fill_color="properties.color",
    pickable=False,
)

# ビュー中心
center = [(minx+maxx)/2, (miny+maxy)/2]


In [25]:
# 前提: DISP_WIDTH / DISP_HEIGHT は既に定義済み
from shapely.geometry import mapping
import pydeck as pdk
from IPython.display import IFrame, display
from time import time
import os

OUT_DIR = "deck_out"   # ノートブックと同じ場所に作る
os.makedirs(OUT_DIR, exist_ok=True)

def render_fixed_square(
    WATER=100, MAP_SIZE=900, ZOOM=15.3, PITCH=55, BEARING=20,
    BASEMAP="osm", filename_prefix="tokyo_water"
):
    # === 1) 水面からの突き出し ===
    tip = (h - WATER).clip(lower=0.0)

    # === 2) 建物GeoJSON ===
    feats = []
    for geom, tip_m in zip(gdf_ll.geometry, tip):
        if geom is None or geom.is_empty or geom.geom_type not in ("Polygon","MultiPolygon"):
            continue
        color = [8,81,156,255] if tip_m > 0 else [158,202,225,180]
        feats.append({
            "type": "Feature",
            "geometry": mapping(geom),
            "properties": {"elev": float(tip_m), "color": color}
        })
    bldg_geojson = {"type": "FeatureCollection", "features": feats}

    layer_bldg = pdk.Layer(
        "GeoJsonLayer", bldg_geojson,
        extruded=True, wireframe=False,
        get_elevation="properties.elev",
        get_fill_color="properties.color",
        get_line_color=[80,80,80,50],
        pickable=True,
    )

    label = pdk.Layer(
        "TextLayer",
        [{"name": f"Water = {WATER} m @ Tokyo Station",
          "lon": center[0], "lat": miny + 0.0006}],
        get_position='[lon, lat]', get_text="name",
        get_size=16, get_color=[10,10,10,220],
        get_alignment_baseline="'bottom'"
    )

    # 3) ベースマップ（OSM/CARTO/Mapboxは前セルの _make_basemap を流用してOK）
    def _make_basemap(kind="osm"):
        url = "https://tile.openstreetmap.org/{z}/{x}/{y}.png" if kind=="osm" else \
              "https://cartodb-basemaps-a.global.ssl.fastly.net/light_all/{z}/{x}/{y}.png"
        return None, pdk.Layer("TileLayer", data=url, min_zoom=0, max_zoom=19, tile_size=256, opacity=1)

    map_style, base_layer = _make_basemap(BASEMAP)
    layers = [base_layer, layer_water, layer_bldg, label] if base_layer else [layer_water, layer_bldg, label]

    # ...（deck = pdk.Deck(...) まではそのまま）
    
    import os, time
    OUT_DIR = "deck_out"
    os.makedirs(OUT_DIR, exist_ok=True)
    
    ts = int(time.time())
    html_name = f"{filename_prefix}_{WATER}m_{MAP_SIZE}px_{ts}.html"
    html_rel  = f"{OUT_DIR}/{html_name}"      # ← ノートブックからの相対パス
    deck.to_html(html_rel, open_browser=False, notebook_display=False)
    
    # 重要：IFrame の src は相対パス（files/ 付けない）
    display(IFrame(src=html_rel, width=DISP_WIDTH, height=DISP_HEIGHT))


In [26]:
render_fixed_square(WATER=100, MAP_SIZE=900)
# 連続で
for w in [5, 15, 20]:
    render_fixed_square(WATER=w, MAP_SIZE=2000)